# 🧪 Class Imbalance Strategies - Adım Adım Test

Her stratejiyi **bağımsız** olarak test et, sonuçları karşılaştır.

## 💡 Kullanım:
1. Setup hücrelerini çalıştır (1-4)
2. İstediğin stratejiyi seç (Strategy 1-5)
3. Sadece o stratejiyi çalıştır (~30-45 dk)
4. Results Comparison hücresini çalıştır
5. Başka bir strateji test et

**Avantaj:** Her stratejiyi ayrı ayrı test edip sonuçları görebilirsin!

## 🔧 SETUP (Her zaman çalıştır)

### 1. Colab Setup

In [ ]:
import sys
import os

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("✅ Google Colab ortamı tespit edildi")
    
    import torch
    if torch.cuda.is_available():
        print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
        print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    else:
        print("⚠️  GPU bulunamadı! Runtime > Change runtime type > GPU seçin")
    
    print("\n📥 Proje indiriliyor...")
    !git clone https://github.com/Aliekinozcetin/Mitre_Attack_TTP_Mapping.git
    os.chdir('Mitre_Attack_TTP_Mapping')
    print(f"✅ Çalışma dizini: {os.getcwd()}")
    
    print("\n📦 Paketler yükleniyor...")
    !pip install -q torch transformers datasets scikit-learn pandas tqdm matplotlib seaborn
    print("✅ Tüm paketler yüklendi")
else:
    print("ℹ️  Yerel ortamda çalışıyorsunuz")

### 2. Import Modules

In [ ]:
import torch
import numpy as np
import json
from datetime import datetime
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

from src.data_loader import prepare_data
from src.model import load_model
from src.train import train_model
from src.evaluate import evaluate_model
from src.strategies import get_strategy_config

print("✅ Modüller yüklendi")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

---

## 🔧 CONFIGURATION

In [ ]:
# Base training configuration
BASE_CONFIG = {
    'model_name': 'bert-base-uncased',
    'batch_size': 16,
    'learning_rate': 2e-5,
    'num_epochs': 3,
    'max_length': 128,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}

# Output directory
OUTPUT_DIR = Path('outputs')
OUTPUT_DIR.mkdir(exist_ok=True)

# Store results from all strategies
all_test_results = {}

print("✅ Konfigürasyon ayarlandı")
print(f"Device: {BASE_CONFIG['device']}")
print(f"Output: {OUTPUT_DIR.absolute()}")

---

## 📊 DATA LOADING

In [ ]:
print("📥 Veri yükleniyor...")

# Load and prepare data
train_dataloader, test_dataloader, num_labels, label_names = prepare_data(
    model_name=BASE_CONFIG['model_name'],
    batch_size=BASE_CONFIG['batch_size'],
    max_length=BASE_CONFIG['max_length']
)

print(f"✅ Veri yüklendi")
print(f"   Train batches: {len(train_dataloader)}")
print(f"   Test batches: {len(test_dataloader)}")
print(f"   Toplam label sayısı: {num_labels}")
print(f"   İlk 5 label: {label_names[:5]}")

---

## 🧪 STRATEGY TESTING

Her strateji bağımsız olarak test edilebilir. İstediğiniz hücreyi çalıştırın.

### Strategy 1: Baseline (Standard BCE Loss)

**Açıklama:** Standart Binary Cross-Entropy loss kullanır. Referans performans için baseline.

**Önerilen Kullanım:** İlk olarak bu stratejiyi çalıştırın (30-45 dakika).

In [ ]:
strategy_name = "baseline"
print(f"\n{'='*60}")
print(f"🧪 STRATEGY 1: Baseline BCE")
print(f"{'='*60}\n")

# Get strategy configuration
config = get_strategy_config(
    strategy_name=strategy_name,
    base_config=BASE_CONFIG,
    num_labels=num_labels,
    train_dataloader=train_dataloader
)

print("📋 Konfigürasyon:")
for key, value in config.items():
    if key != 'pos_weight':
        print(f"   {key}: {value}")

# Load model
print("\n🔧 Model yükleniyor...")
model = load_model(
    model_name=config['model_name'],
    num_labels=config['num_labels'],
    loss_type=config.get('loss_type', 'bce'),
    alpha=config.get('alpha'),
    gamma=config.get('gamma'),
    pos_weight=config.get('pos_weight')
).to(config['device'])

# Train model
print("\n🚀 Eğitim başlıyor...")
training_history = train_model(
    model=model,
    train_dataloader=config['train_dataloader'],
    num_epochs=config['num_epochs'],
    learning_rate=config['learning_rate'],
    device=config['device']
)

# Evaluate model
print("\n📊 Test seti değerlendiriliyor...")
test_results = evaluate_model(
    model=model,
    test_dataloader=config['test_dataloader'],
    label_names=label_names,
    device=config['device']
)

# Store results
all_test_results[strategy_name] = {
    'config': {k: v for k, v in config.items() if k not in ['train_dataloader', 'test_dataloader', 'pos_weight', 'device']},
    'results': test_results
}

# Display results
print("\n" + "="*60)
print(f"✅ STRATEGY 1 TAMAMLANDI: {strategy_name}")
print("="*60)
print(f"\n📈 Sonuçlar:")
for metric, value in test_results.items():
    if isinstance(value, (int, float)):
        print(f"   {metric}: {value:.4f}")
print("\n")

### Strategy 2: Weighted BCE Loss

**Açıklama:** Her label için frekans bazlı ağırlık hesaplar (pos_weight=458 for rare labels). Class imbalance için en etkili yöntem.

**Önerilen Kullanım:** Baseline'dan sonra bu stratejiyi test edin. F1 > 0.15 ise, diğer stratejileri test etmeye gerek yok.

In [ ]:
strategy_name = "weighted_bce"
print(f"\n{'='*60}")
print(f"🧪 STRATEGY 2: Weighted BCE")
print(f"{'='*60}\n")

# Get strategy configuration
config = get_strategy_config(
    strategy_name=strategy_name,
    base_config=BASE_CONFIG,
    num_labels=num_labels,
    train_dataloader=train_dataloader
)

print("📋 Konfigürasyon:")
for key, value in config.items():
    if key == 'pos_weight':
        print(f"   pos_weight: tensor shape {value.shape} (min={value.min():.2f}, max={value.max():.2f})")
    elif key not in ['train_dataloader', 'test_dataloader', 'device']:
        print(f"   {key}: {value}")

# Load model
print("\n🔧 Model yükleniyor...")
model = load_model(
    model_name=config['model_name'],
    num_labels=config['num_labels'],
    loss_type=config.get('loss_type', 'bce'),
    alpha=config.get('alpha'),
    gamma=config.get('gamma'),
    pos_weight=config.get('pos_weight')
).to(config['device'])

# Train model
print("\n🚀 Eğitim başlıyor...")
training_history = train_model(
    model=model,
    train_dataloader=config['train_dataloader'],
    num_epochs=config['num_epochs'],
    learning_rate=config['learning_rate'],
    device=config['device']
)

# Evaluate model
print("\n📊 Test seti değerlendiriliyor...")
test_results = evaluate_model(
    model=model,
    test_dataloader=config['test_dataloader'],
    label_names=label_names,
    device=config['device']
)

# Store results
all_test_results[strategy_name] = {
    'config': {k: v for k, v in config.items() if k not in ['train_dataloader', 'test_dataloader', 'pos_weight', 'device']},
    'results': test_results
}

# Display results
print("\n" + "="*60)
print(f"✅ STRATEGY 2 TAMAMLANDI: {strategy_name}")
print("="*60)
print(f"\n📈 Sonuçlar:")
for metric, value in test_results.items():
    if isinstance(value, (int, float)):
        print(f"   {metric}: {value:.4f}")
print("\n")

### Strategy 3: Focal Loss (γ=2, α=0.25)

**Açıklama:** Focal Loss with moderate focusing (γ=2). Hard örneklere odaklanır.

**Önerilen Kullanım:** Weighted BCE başarısız olursa deneyin.

In [ ]:
strategy_name = "focal_loss_weak"
print(f"\n{'='*60}")
print(f"🧪 STRATEGY 3: Focal Loss (γ=2)")
print(f"{'='*60}\n")

# Get strategy configuration
config = get_strategy_config(
    strategy_name=strategy_name,
    base_config=BASE_CONFIG,
    num_labels=num_labels,
    train_dataloader=train_dataloader
)

print("📋 Konfigürasyon:")
for key, value in config.items():
    if key not in ['train_dataloader', 'test_dataloader', 'pos_weight', 'device']:
        print(f"   {key}: {value}")

# Load model
print("\n🔧 Model yükleniyor...")
model = load_model(
    model_name=config['model_name'],
    num_labels=config['num_labels'],
    loss_type=config.get('loss_type', 'bce'),
    alpha=config.get('alpha'),
    gamma=config.get('gamma'),
    pos_weight=config.get('pos_weight')
).to(config['device'])

# Train model
print("\n🚀 Eğitim başlıyor...")
training_history = train_model(
    model=model,
    train_dataloader=config['train_dataloader'],
    num_epochs=config['num_epochs'],
    learning_rate=config['learning_rate'],
    device=config['device']
)

# Evaluate model
print("\n📊 Test seti değerlendiriliyor...")
test_results = evaluate_model(
    model=model,
    test_dataloader=config['test_dataloader'],
    label_names=label_names,
    device=config['device']
)

# Store results
all_test_results[strategy_name] = {
    'config': {k: v for k, v in config.items() if k not in ['train_dataloader', 'test_dataloader', 'pos_weight', 'device']},
    'results': test_results
}

# Display results
print("\n" + "="*60)
print(f"✅ STRATEGY 3 TAMAMLANDI: {strategy_name}")
print("="*60)
print(f"\n📈 Sonuçlar:")
for metric, value in test_results.items():
    if isinstance(value, (int, float)):
        print(f"   {metric}: {value:.4f}")
print("\n")

### Strategy 4: Focal Loss (γ=5, α=0.25)

**Açıklama:** Focal Loss with strong focusing (γ=5). Çok hard örneklere odaklanır.

**Önerilen Kullanım:** Focal γ=2 başarılı olursa daha güçlü versiyon için deneyin.

In [ ]:
strategy_name = "focal_loss_strong"
print(f"\n{'='*60}")
print(f"🧪 STRATEGY 4: Focal Loss (γ=5)")
print(f"{'='*60}\n")

# Get strategy configuration
config = get_strategy_config(
    strategy_name=strategy_name,
    base_config=BASE_CONFIG,
    num_labels=num_labels,
    train_dataloader=train_dataloader
)

print("📋 Konfigürasyon:")
for key, value in config.items():
    if key not in ['train_dataloader', 'test_dataloader', 'pos_weight', 'device']:
        print(f"   {key}: {value}")

# Load model
print("\n🔧 Model yükleniyor...")
model = load_model(
    model_name=config['model_name'],
    num_labels=config['num_labels'],
    loss_type=config.get('loss_type', 'bce'),
    alpha=config.get('alpha'),
    gamma=config.get('gamma'),
    pos_weight=config.get('pos_weight')
).to(config['device'])

# Train model
print("\n🚀 Eğitim başlıyor...")
training_history = train_model(
    model=model,
    train_dataloader=config['train_dataloader'],
    num_epochs=config['num_epochs'],
    learning_rate=config['learning_rate'],
    device=config['device']
)

# Evaluate model
print("\n📊 Test seti değerlendiriliyor...")
test_results = evaluate_model(
    model=model,
    test_dataloader=config['test_dataloader'],
    label_names=label_names,
    device=config['device']
)

# Store results
all_test_results[strategy_name] = {
    'config': {k: v for k, v in config.items() if k not in ['train_dataloader', 'test_dataloader', 'pos_weight', 'device']},
    'results': test_results
}

# Display results
print("\n" + "="*60)
print(f"✅ STRATEGY 4 TAMAMLANDI: {strategy_name}")
print("="*60)
print(f"\n📈 Sonuçlar:")
for metric, value in test_results.items():
    if isinstance(value, (int, float)):
        print(f"   {metric}: {value:.4f}")
print("\n")

### Strategy 5: Top-100 Labels Subset

**Açıklama:** Sadece en sık 100 label ile eğitim. Model'in öğrenme kapasitesini test etmek için.

**Önerilen Kullanım:** Tüm stratejiler başarısız olursa, modelin genel öğrenme kabiliyetini kontrol etmek için deneyin.

In [ ]:
strategy_name = "top_100"
print(f"\n{'='*60}")
print(f"🧪 STRATEGY 5: Top-100 Labels")
print(f"{'='*60}\n")

# Get strategy configuration
config = get_strategy_config(
    strategy_name=strategy_name,
    base_config=BASE_CONFIG,
    num_labels=num_labels,
    train_dataloader=train_dataloader
)

print("📋 Konfigürasyon:")
for key, value in config.items():
    if key not in ['train_dataloader', 'test_dataloader', 'pos_weight', 'device']:
        print(f"   {key}: {value}")

# Load model
print("\n🔧 Model yükleniyor...")
model = load_model(
    model_name=config['model_name'],
    num_labels=config['num_labels'],
    loss_type=config.get('loss_type', 'bce'),
    alpha=config.get('alpha'),
    gamma=config.get('gamma'),
    pos_weight=config.get('pos_weight')
).to(config['device'])

# Train model
print("\n🚀 Eğitim başlıyor...")
training_history = train_model(
    model=model,
    train_dataloader=config['train_dataloader'],
    num_epochs=config['num_epochs'],
    learning_rate=config['learning_rate'],
    device=config['device']
)

# Evaluate model
print("\n📊 Test seti değerlendiriliyor...")
test_results = evaluate_model(
    model=model,
    test_dataloader=config['test_dataloader'],
    label_names=label_names[:config['num_labels']],  # Only top-100 labels
    device=config['device']
)

# Store results
all_test_results[strategy_name] = {
    'config': {k: v for k, v in config.items() if k not in ['train_dataloader', 'test_dataloader', 'pos_weight', 'device']},
    'results': test_results
}

# Display results
print("\n" + "="*60)
print(f"✅ STRATEGY 5 TAMAMLANDI: {strategy_name}")
print("="*60)
print(f"\n📈 Sonuçlar:")
for metric, value in test_results.items():
    if isinstance(value, (int, float)):
        print(f"   {metric}: {value:.4f}")
print("\n")

---

## 📊 RESULTS COMPARISON

Çalıştırdığınız stratejilerin sonuçlarını karşılaştırmak için bu hücreyi çalıştırın.

In [ ]:
if len(all_test_results) == 0:
    print("⚠️ Henüz hiçbir strateji test edilmedi!")
else:
    print("\n" + "="*80)
    print("📊 STRATEGY COMPARISON RESULTS")
    print("="*80 + "\n")
    
    # Create comparison table
    comparison_data = []
    for strategy_name, data in all_test_results.items():
        results = data['results']
        comparison_data.append({
            'Strategy': strategy_name,
            'F1 (Top-5)': results.get('f1_score_top5', 0),
            'F1 (Top-10)': results.get('f1_score_top10', 0),
            'Precision': results.get('precision', 0),
            'Recall': results.get('recall', 0),
            'Hamming Loss': results.get('hamming_loss', 0)
        })
    
    df_comparison = pd.DataFrame(comparison_data)
    df_comparison = df_comparison.sort_values('F1 (Top-5)', ascending=False)
    
    print(df_comparison.to_string(index=False))
    print("\n" + "="*80)
    
    # Find best strategy
    best_strategy = df_comparison.iloc[0]['Strategy']
    best_f1 = df_comparison.iloc[0]['F1 (Top-5)']
    
    print(f"\n🏆 EN İYİ STRATEJİ: {best_strategy}")
    print(f"   F1 Score (Top-5): {best_f1:.4f}")
    
    # Compare to baseline if exists
    if 'baseline' in all_test_results:
        baseline_f1 = all_test_results['baseline']['results'].get('f1_score_top5', 0)
        improvement = ((best_f1 - baseline_f1) / baseline_f1 * 100) if baseline_f1 > 0 else 0
        print(f"   Baseline'a göre iyileştirme: {improvement:+.2f}%")
    
    # Plot comparison
    if len(all_test_results) > 1:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # F1 scores comparison
        strategies = df_comparison['Strategy'].tolist()
        f1_top5 = df_comparison['F1 (Top-5)'].tolist()
        f1_top10 = df_comparison['F1 (Top-10)'].tolist()
        
        x = np.arange(len(strategies))
        width = 0.35
        
        axes[0].bar(x - width/2, f1_top5, width, label='Top-5', alpha=0.8)
        axes[0].bar(x + width/2, f1_top10, width, label='Top-10', alpha=0.8)
        axes[0].set_xlabel('Strategy')
        axes[0].set_ylabel('F1 Score')
        axes[0].set_title('F1 Score Comparison')
        axes[0].set_xticks(x)
        axes[0].set_xticklabels(strategies, rotation=45, ha='right')
        axes[0].legend()
        axes[0].grid(axis='y', alpha=0.3)
        
        # Precision-Recall comparison
        precision = df_comparison['Precision'].tolist()
        recall = df_comparison['Recall'].tolist()
        
        axes[1].bar(x - width/2, precision, width, label='Precision', alpha=0.8)
        axes[1].bar(x + width/2, recall, width, label='Recall', alpha=0.8)
        axes[1].set_xlabel('Strategy')
        axes[1].set_ylabel('Score')
        axes[1].set_title('Precision & Recall Comparison')
        axes[1].set_xticks(x)
        axes[1].set_xticklabels(strategies, rotation=45, ha='right')
        axes[1].legend()
        axes[1].grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    print("\n")

---

## 💾 SAVE RESULTS

Sonuçları kaydetmek için bu hücreyi çalıştırın.

In [ ]:
if len(all_test_results) == 0:
    print("⚠️ Kaydedilecek sonuç yok!")
else:
    # Save results to JSON
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    results_file = OUTPUT_DIR / f'strategy_comparison_{timestamp}.json'
    
    with open(results_file, 'w') as f:
        json.dump(all_test_results, f, indent=2)
    
    print(f"✅ Sonuçlar kaydedildi: {results_file}")
    
    # Save comparison table to CSV
    comparison_data = []
    for strategy_name, data in all_test_results.items():
        results = data['results']
        comparison_data.append({
            'Strategy': strategy_name,
            'F1_Top5': results.get('f1_score_top5', 0),
            'F1_Top10': results.get('f1_score_top10', 0),
            'Precision': results.get('precision', 0),
            'Recall': results.get('recall', 0),
            'Hamming_Loss': results.get('hamming_loss', 0)
        })
    
    df_comparison = pd.DataFrame(comparison_data)
    csv_file = OUTPUT_DIR / f'strategy_comparison_{timestamp}.csv'
    df_comparison.to_csv(csv_file, index=False)
    
    print(f"✅ CSV tablosu kaydedildi: {csv_file}")
    
    # Download files if on Colab
    try:
        from google.colab import files
        files.download(str(results_file))
        files.download(str(csv_file))
        print("✅ Dosyalar indirildi")
    except ImportError:
        print("💡 Colab değil, dosyalar sadece kaydedildi")

---

## 📝 NOTLAR VE ÖNERİLER

**Test Sırası:**
1. **Baseline** - Her zaman ilk olarak çalıştırın (referans performans)
2. **Weighted BCE** - En umut verici strateji, F1 > 0.15 bekleniyor
3. Weighted BCE başarılı olursa → Bitti, kullan!
4. Weighted BCE başarısız olursa → **Focal Loss (γ=2)** dene
5. Hala kötüyse → **Focal Loss (γ=5)** dene
6. Hepsi başarısız olursa → **Top-100** ile model öğrenme kapasitesini kontrol et

**Başarı Kriterleri:**
- F1 (Top-5) > 0.15 → İyi performans
- F1 (Top-5) > 0.25 → Çok iyi performans
- F1 (Top-5) < 0.10 → Strateji işe yaramadı

**Süre:**
- Her strateji ~30-45 dakika sürer
- Modular yaklaşım sayesinde istediğiniz zaman durup sonuçlara bakabilirsiniz

**Hatırlatmalar:**
- Herbir strateji hücresi bağımsızdır, istediğiniz sırada çalıştırabilirsiniz
- Sonuçlar `all_test_results` dictionary'sinde saklanır
- Comparison hücresini dilediğiniz zaman çalıştırıp ara sonuçlara bakabilirsiniz